In [1]:
%gui qt

from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_FACE
from OCC.Core.TopoDS import topods
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Plane

import numpy as np

# Start display without blocking
display, start_display, add_menu, add_function_to_menu = init_display()

display.EraseAll()
display.View_Iso()
display.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
from OCC.Extend.DataExchange import read_step_file
shape = read_step_file("../data/0444-1 ANGLED.step")
# shape = read_step_file("../data/ncTest.step")
# shape = read_step_file("../data/TestEA.step")
# shape = read_step_file("../data/TestUEA.step")
# shape = read_step_file("../data/TestPFC.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [3]:
# Helper: draw an axis arrow
def draw_axis_arrow(display, origin, direction, scale=100, color='BLACK'):
    end = gp_Pnt(
        origin.X() + direction.X() * scale,
        origin.Y() + direction.Y() * scale,
        origin.Z() + direction.Z() * scale,
    )
    edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
    display.DisplayShape(edge, color=color, update=False)


In [4]:
def compute_obb_and_local_axes(solid):
    from OCC.Core.Bnd import Bnd_OBB
    from OCC.Core.BRepBndLib import brepbndlib
    from OCC.Core.gp import gp_Vec

    # Compute OBB
    obb = Bnd_OBB()
    brepbndlib.AddOBB(solid, obb, True, True, False)

    # Get raw data
    center = obb.Center()
    axes = [obb.XDirection(), obb.YDirection(), obb.ZDirection()]
    half_extents = [obb.XHSize(), obb.YHSize(), obb.ZHSize()]

    # Sort half-extents to identify logical axes
    sorted_indices = np.argsort(half_extents)
    web_idx, flange_idx, length_idx = sorted_indices

    # Assign named axes + extents
    xaxis = gp_Vec(axes[web_idx])
    yaxis = gp_Vec(axes[flange_idx])
    zaxis = gp_Vec(axes[length_idx])

    he_X = half_extents[web_idx]
    he_Y = half_extents[flange_idx]
    he_Z = half_extents[length_idx]

    return {
        'center': center,
        'xaxis': xaxis,
        'yaxis': yaxis,
        'zaxis': zaxis,
        'he_X': he_X,
        'he_Y': he_Y,
        'he_Z': he_Z,
        'axes': (xaxis, yaxis, zaxis),
        'half_extents': (he_X, he_Y, he_Z)
    }


In [5]:
def draw_plane(display, origin, normal, size=50, color="MAGENTA"):
    """
    Draws a square plane centered at `origin`, normal to `normal` axis vector.
    
    Parameters:
        display -- OCC display
        origin  -- gp_Pnt (plane center)
        normal  -- gp_Vec (plane normal)
        size    -- length of plane sides
        color   -- display color
    """
    from OCC.Core.gp import gp_Vec, gp_Pnt
    import numpy as np

    # Convert normal to numpy
    n = np.array([normal.X(), normal.Y(), normal.Z()])
    n = n / np.linalg.norm(n)

    # Choose a reference vector not colinear with `n`
    ref = np.array([0, 0, 1])
    if np.abs(np.dot(n, ref)) > 0.99:
        ref = np.array([1, 0, 0])

    # Compute local u, v plane axes
    u = np.cross(n, ref); u = u / np.linalg.norm(u)
    v = np.cross(n, u);   v = v / np.linalg.norm(v)

    # Convert origin to numpy
    o = np.array([origin.X(), origin.Y(), origin.Z()])
    s = size / 2.0

    # Create four corner points
    pts = [
        gp_Pnt(*(o + s * ( u + v))),
        gp_Pnt(*(o + s * (-u + v))),
        gp_Pnt(*(o + s * (-u - v))),
        gp_Pnt(*(o + s * ( u - v))),
    ]

    # Create edges between corners
    for i in range(4):
        e = BRepBuilderAPI_MakeEdge(pts[i], pts[(i+1)%4]).Edge()
        display.DisplayShape(e, color=color, update=False)


In [6]:
def cut_section_area(solid, origin, normal):
    plane = gp_Pln(origin, gp_Dir(normal))
    section = BRepAlgoAPI_Section(solid, plane)
    section.ComputePCurveOn1(True)
    section.Approximation(True)
    section.Build()

    if not section.IsDone():
        return 0.0

    wire_builder = BRepBuilderAPI_MakeWire()
    exp = TopExp_Explorer(section.Shape(), TopAbs_EDGE)
    while exp.More():
        wire_builder.Add(topods.Edge(exp.Current()))
        exp.Next()

    if wire_builder.IsDone():
        face = BRepBuilderAPI_MakeFace(wire_builder.Wire()).Face()
        props = GProp_GProps()
        brepgprop.SurfaceProperties(face, props)
        return props.Mass()
    return 0.0

In [7]:
def ensure_correct_zaxis_for_angle(solid, center, zaxis, half_length):
    origin_start = gp_Pnt(
        center.X() - half_length * zaxis.X(),
        center.Y() - half_length * zaxis.Y(),
        center.Z() - half_length * zaxis.Z(),
    )
    origin_end = gp_Pnt(
        center.X() + half_length * zaxis.X(),
        center.Y() + half_length * zaxis.Y(),
        center.Z() + half_length * zaxis.Z(),
    )

    area_start = cut_section_area(solid, origin_start, zaxis)
    area_end = cut_section_area(solid, origin_end, zaxis)

    print(f"\nSection area at start: {area_start:.10f}")
    print(f"Section area at end:   {area_end:.10f}")

    if area_end > area_start:
        print("\n🔁 Flipping z-axis for angle profile")
        return gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z())
    return zaxis

In [8]:
def cut_section_at_start(solid, center, zaxis, he_Z, display=None, show_plane=True):
    """
    Cuts a cross-section at the start of the beam (negative Z direction from center).

    Parameters:
        solid   -- The STEP solid (TopoDS_Shape)
        center  -- gp_Pnt of the OBB center
        zaxis   -- gp_Vec (local length axis)
        he_Z    -- Half-length (float)
        display -- OCC Display instance (optional, for visualisation)
        show_plane -- If True, shows cutting plane

    Returns:
        face  -- The section face (TopoDS_Face)
        wire  -- The section wire (TopoDS_Wire)
        area  -- Section area (float)
        origin -- gp_Pnt of the cutting plane origin
    """
    # Compute start origin: center - he_Z * zaxis
    origin_vec = np.array([center.X(), center.Y(), center.Z()]) - he_Z * np.array([zaxis.X(), zaxis.Y(), zaxis.Z()])
    origin = gp_Pnt(*origin_vec)

    # Plane normal is aligned with zaxis (beam length direction)
    plane = gp_Pln(origin, gp_Dir(zaxis))

    # Perform section
    sec = BRepAlgoAPI_Section(solid, plane)
    sec.ComputePCurveOn1(True)
    sec.Approximation(True)
    sec.Build()

    # Build wire
    exp_e = TopExp_Explorer(sec.Shape(), TopAbs_EDGE)
    wb = BRepBuilderAPI_MakeWire()
    while exp_e.More():
        wb.Add(topods.Edge(exp_e.Current()))
        exp_e.Next()
    wire = wb.Wire()

    # Create face and compute area
    face = BRepBuilderAPI_MakeFace(wire).Face()
    gp2 = GProp_GProps()
    brepgprop.SurfaceProperties(face, gp2)
    area = gp2.Mass()

    # Optionally show plane
    if display and show_plane:
        draw_plane(display, origin, zaxis, size=50, color="RED")
        display.DisplayShape(origin, color="RED", update=False)
        display.DisplayShape(wire, color='CYAN', update=True)

    return face, wire, area, origin


In [9]:
def draw_obb(display, center, xaxis, yaxis, zaxis, he_X, he_Y, he_Z, color="WHITE"):
    # Convert center to numpy array
    c = np.array([center.X(), center.Y(), center.Z()])

    # Convert each axis to numpy vector
    xvec = np.array([xaxis.X(), xaxis.Y(), xaxis.Z()])
    yvec = np.array([yaxis.X(), yaxis.Y(), yaxis.Z()])
    zvec = np.array([zaxis.X(), zaxis.Y(), zaxis.Z()])

    # Generate 8 corner points
    corners = []
    for dx in (-1, 1):
        for dy in (-1, 1):
            for dz in (-1, 1):
                offset = dx * he_X * xvec + dy * he_Y * yvec + dz * he_Z * zvec
                pt = gp_Pnt(*(c + offset))
                corners.append(pt)

    # Define OBB wireframe edges (12 edges)
    edges = [
        (0, 1), (0, 2), (0, 4),
        (1, 3), (1, 5),
        (2, 3), (2, 6),
        (3, 7),
        (4, 5), (4, 6),
        (5, 7),
        (6, 7)
    ]

    # Draw edges
    for i, j in edges:
        edge = BRepBuilderAPI_MakeEdge(corners[i], corners[j]).Edge()
        display.DisplayShape(edge, color=color, update=False)


In [10]:

def fingerprint_shape(solid, obb_dims):
    """
    Analyze face normals and areas to guess profile type: Beam, Channel, Angle, Unknown
    """
    face_areas = []
    face_normals = []

    exp = TopExp_Explorer(solid, TopAbs_FACE)
    while exp.More():
        face = topods.Face(exp.Current())
        surf = BRepAdaptor_Surface(face)
        if surf.GetType() != GeomAbs_Plane:
            exp.Next()
            continue

        # Normal
        n = surf.Plane().Axis().Direction()
        n = np.array([n.X(), n.Y(), n.Z()], dtype=float)
        n /= np.linalg.norm(n)

        # Area
        gp = GProp_GProps()
        brepgprop.SurfaceProperties(face, gp)
        area = gp.Mass()

        face_normals.append(n)
        face_areas.append(area)
        exp.Next()

    # Sort OBB dimensions
    dims = sorted(obb_dims)  # [flange_thickness, web_depth, length]
    thickness, height, length = dims
    slenderness = length / height if height else 0
    aspect_ratio = height / thickness if thickness else 0

    # Area stats
    largest = max(face_areas) if face_areas else 0
    large_faces = [a for a in face_areas if a > 0.8 * largest]

    print("\n🔎 Shape fingerprint debug:")
    print(f"  OBB dimensions (sorted): {np.round(dims,1)}")
    print(f"  Slenderness ratio (L/H): {slenderness:.2f}")
    print(f"  Aspect ratio (H/t):      {aspect_ratio:.2f}")
    print(f"  Total planar faces:      {len(face_areas)}")
    print(f"  Large planar faces:      {len(large_faces)}")
    print(f"  Max area:                {max(face_areas):.1f}" if face_areas else "  Max area: —")
    print(f"  Large face areas:        {[round(a,1) for a in large_faces]}")
    
    # Fingerprint logic
    
    # --- Check for Beam ---
    if len(face_areas) >= 10 and slenderness > 4 and max(face_areas) > 100000:
        return "I"
    
    # --- Check for Angles ---
    elif len(large_faces) == 2 and slenderness > 6:
        a1, a2 = large_faces
        area_ratio = max(a1, a2) / min(a1, a2)
        if area_ratio < 1.1:
            return "L"
        else:
            return "L"

    # --- Check for Channel ---
    elif len(face_areas) >= 6 and slenderness > 6 and max(face_areas) > 50000:
        return "U"
        
    # --- Fallback ---
    return "Unknown"


In [11]:
def color_solid_by_profile(display, solid, profile_type):
    color_map = {
        "I": "BLUE",
        "U": "ORANGE",
        "L": "CYAN",
        "Unknown": "RED"  # Red for problematic/unknown profiles
    }
    color = color_map.get(profile_type, "WHITE")
    display.DisplayShape(solid, color=color, update=True)


In [12]:
# Global dictionary to track labels
labels = {}

def label_2d_text(display, label_id, position_gp_pnt, text, height=12):
    """
    Display a named 2D label at a 3D position and store its metadata for later modification.
    """
    display.DisplayMessage(position_gp_pnt, text, height=height)
    labels[label_id] = {
        "position": position_gp_pnt,
        "text": text,
        "height": height
    }



In [13]:
# from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
# from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
# from OCC.Core.TopExp import TopExp_Explorer
# from OCC.Core.TopAbs import TopAbs_EDGE
# from OCC.Core.TopoDS import topods
# from OCC.Core.gp import gp_Pln, gp_Dir, gp_Pnt
# import numpy as np


def cut_section_face(solid, origin, normal):
    """
    Cut a section face through the solid at a given origin and normal.
    Returns a list of 3D points.
    """
    plane = gp_Pln(origin, gp_Dir(normal))
    section = BRepAlgoAPI_Section(solid, plane)
    section.ComputePCurveOn1(True)
    section.Approximation(True)
    section.Build()

    exp = TopExp_Explorer(section.Shape(), TopAbs_EDGE)
    wire_builder = BRepBuilderAPI_MakeWire()
    while exp.More():
        wire_builder.Add(topods.Edge(exp.Current()))
        exp.Next()

    if not wire_builder.IsDone():
        return []

    face = BRepBuilderAPI_MakeFace(wire_builder.Wire()).Face()

    # Sample points on edges
    points = []
    exp = TopExp_Explorer(face, TopAbs_EDGE)
    while exp.More():
        edge = topods.Edge(exp.Current())
        curve_adapt = BRepAdaptor_Curve(edge)
        p1 = curve_adapt.Value(curve_adapt.FirstParameter())
        p2 = curve_adapt.Value(curve_adapt.LastParameter())
        points.append(np.array([p1.X(), p1.Y(), p1.Z()]))
        points.append(np.array([p2.X(), p2.Y(), p2.Z()]))
        exp.Next()

    return points


# def should_flip_angle_zaxis(solid, center, zaxis, he_Z, xaxis, yaxis):
#     """
#     Determine if the angle profile's Z axis needs to be flipped.
#     Returns True if the current Z axis is pointing toward the "wrong" end.
#     """
#     start_origin = gp_Pnt(*(np.array([center.X(), center.Y(), center.Z()]) - he_Z * np.array([zaxis.X(), zaxis.Y(), zaxis.Z()])))
#     end_origin = gp_Pnt(*(np.array([center.X(), center.Y(), center.Z()]) + he_Z * np.array([zaxis.X(), zaxis.Y(), zaxis.Z()])))

#     start_pts = cut_section_face(solid, start_origin, zaxis)
#     end_pts = cut_section_face(solid, end_origin, -zaxis)

#     if not start_pts or not end_pts:
#         return False  # fallback

#     def get_l_corner_score(pts, xaxis, yaxis, origin):
#         """Project points to local plane and score based on corner location"""
#         local_pts = []
#         ox = np.array([xaxis.X(), xaxis.Y(), xaxis.Z()])
#         oy = np.array([yaxis.X(), yaxis.Y(), yaxis.Z()])
#         o = np.array([origin.X(), origin.Y(), origin.Z()])
#         for p in pts:
#             rel = p - o
#             px = np.dot(rel, ox)
#             py = np.dot(rel, oy)
#             local_pts.append([px, py])
#         local_pts = np.array(local_pts)
#         if not len(local_pts):
#             return 0
#         bbox = np.array([np.min(local_pts, axis=0), np.max(local_pts, axis=0)])
#         # score is negative of distance of the lower-left corner from origin
#         return -np.linalg.norm(bbox[0])

#     start_score = get_l_corner_score(start_pts, xaxis, yaxis, start_origin)
#     end_score = get_l_corner_score(end_pts, xaxis, yaxis, end_origin)

#     return end_score > start_score  # if end is "more L-like", flip




In [14]:
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_FACE
from OCC.Core.TopoDS import topods
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
import numpy as np


def adjust_zaxis_for_angle_by_end_face_area(solid, center, zaxis, half_length):
    """
    Use planar sections to detect the correct orientation of an L-profile.
    Chooses the end face with larger area and adjusts Z axis accordingly.
    Returns: (zaxis: gp_Dir, was_flipped: bool)
    """
    def make_section_face(offset):
        plane_origin = gp_Pnt(
            center.X() + offset * zaxis.X(),
            center.Y() + offset * zaxis.Y(),
            center.Z() + offset * zaxis.Z()
        )
        plane = gp_Pln(plane_origin, gp_Dir(zaxis))
        section = BRepAlgoAPI_Section(solid, plane)
        section.ComputePCurveOn1(True)
        section.Approximation(True)
        section.Build()

        wire_builder = BRepBuilderAPI_MakeWire()
        exp = TopExp_Explorer(section.Shape(), TopAbs_EDGE)
        while exp.More():
            wire_builder.Add(topods.Edge(exp.Current()))
            exp.Next()

        face = BRepBuilderAPI_MakeFace(wire_builder.Wire()).Face()
        return face

    def get_face_area(face):
        props = GProp_GProps()
        brepgprop.SurfaceProperties(face, props)
        return props.Mass()

    face1 = make_section_face(half_length)
    face2 = make_section_face(-half_length)

    area1 = get_face_area(face1)
    area2 = get_face_area(face2)

    print(f"End face area comparison → Forward: {area1:.1f}, Reverse: {area2:.1f}")

    if area1 >= area2:
        return zaxis, False
    else:
        return zaxis.Reversed(), True


In [15]:
def compute_dstv_origins_and_axes(center, half_extents, xaxis, yaxis, zaxis, was_flipped):
    """
    Compute DSTV origins (V, O, U) and face axes using local coordinate system.
    """
    he_X, he_Y, he_Z = half_extents

    # Center point
    cx, cy, cz = center.X(), center.Y(), center.Z()

    # Use xaxis/yaxis/zaxis directly as gp_Vec
    origin_v = gp_Pnt(cx + he_Z * zaxis.X() + he_Y * yaxis.X() - he_X * xaxis.X(),
                      cy + he_Z * zaxis.Y() + he_Y * yaxis.Y() - he_X * xaxis.Y(),
                      cz + he_Z * zaxis.Z() + he_Y * yaxis.Z() - he_X * xaxis.Z())

    origin_o = gp_Pnt(cx + he_Z * zaxis.X() - he_Y * yaxis.X() + he_X * xaxis.X(),
                      cy + he_Z * zaxis.Y() - he_Y * yaxis.Y() + he_X * xaxis.Y(),
                      cz + he_Z * zaxis.Z() - he_Y * yaxis.Z() + he_X * xaxis.Z())

    origin_u = gp_Pnt(cx + he_Z * zaxis.X() - he_Y * yaxis.X() - he_X * xaxis.X(),
                      cy + he_Z * zaxis.Y() - he_Y * yaxis.Y() - he_X * xaxis.Y(),
                      cz + he_Z * zaxis.Z() - he_Y * yaxis.Z() - he_X * xaxis.Z())

    if profile_type == "L" and was_flipped:
        # use reversed Z for correct DSTV orientation
        vec_z = gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z())
    else:
        vec_z = zaxis
    
    face_axes = {
        'V': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), yaxis),
        'O': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), xaxis),
        'U': (gp_Vec(-zaxis.X(), -zaxis.Y(), -zaxis.Z()), vec_z)
    }


    origins = {
        'V': origin_v,
        'O': origin_o,
        'U': origin_u
    }

    return origins, face_axes


In [16]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeVertex

def visualize_dstv_origins(display, origins, face_axes):
    """
    Visualize DSTV corner origins (V, O, U) with color-coded points and axis arrows.
    """
    color_map = {
        'V': 'MAGENTA',
        'O': 'CYAN',
        'U': 'ORANGE'
    }

    for name, pt in origins.items():
        color = color_map.get(name, 'WHITE')

        # Draw point marker
        vertex = BRepBuilderAPI_MakeVertex(pt).Vertex()
        display.DisplayShape(vertex, color=color, update=False)

        # Draw local face axes
        x_dir, y_dir = face_axes[name]
        draw_axis_arrow(display, pt, x_dir, 50, color='RED')    # X axis
        draw_axis_arrow(display, pt, y_dir, 50, color='GREEN')  # Y axis

    display.FitAll()


In [17]:
# Display solid
display.DisplayShape(solid, update=True)

# Compute OBB
obb_data = compute_obb_and_local_axes(solid)

center       = obb_data['center']
xaxis        = obb_data['xaxis']
yaxis        = obb_data['yaxis']
zaxis        = obb_data['zaxis']
he_X         = obb_data['he_X']
he_Y         = obb_data['he_Y']
he_Z         = obb_data['he_Z']
half_extents = [he_X, he_Y, he_Z]
obb_dims     = [2*he_X, 2*he_Y, 2*he_Z]

print("OBB center:", center.X(), center.Y(), center.Z())

# Draw bounding box
draw_obb(display,
         center=obb_data['center'],
         xaxis=obb_data['xaxis'],
         yaxis=obb_data['yaxis'],
         zaxis=obb_data['zaxis'],
         he_X=obb_data['he_X'],
         he_Y=obb_data['he_Y'],
         he_Z=obb_data['he_Z'],
         color="YELLOW")

# Identify profile type
profile_type = fingerprint_shape(shape, obb_dims)
print(f"\nProfile Type is : {profile_type}")
# check orietation in z axis for angle
was_flipped = False
if profile_type == "L":
    print("Checking angle orientation")
    zaxis, was_flipped = adjust_zaxis_for_angle_by_end_face_area(solid, center, zaxis, he_Z)
    print(f"Flipped Z-Axis : {was_flipped}")
    # Shift center if we flipped Z
    if was_flipped:
        center = gp_Pnt(
            center.X() - 2 * he_Z * zaxis.X(),
            center.Y() - 2 * he_Z * zaxis.Y(),
            center.Z() - 2 * he_Z * zaxis.Z()
        )
    
# Label of identified section type
start_face_center = gp_Pnt(
    center.X() - he_Z * zaxis.X(),
    center.Y() - he_Z * zaxis.Y(),
    center.Z() - he_Z * zaxis.Z()
)

label_id = f"profile_label_{profile_type}"
label_2d_text(display, label_id, start_face_center, f"{profile_type} SECTION", height=24)

print("Determining NC1 Faces and Origin")
# Display DSTV origins and faces
origins, face_axes = compute_dstv_origins_and_axes(center, half_extents, xaxis, yaxis, zaxis, was_flipped)
visualize_dstv_origins(display, origins, face_axes)


display.FitAll()

OBB center: 0.0 -570.0000000000001 -987.2689603142607

🔎 Shape fingerprint debug:
  OBB dimensions (sorted): [ 203.2  203.6 2280. ]
  Slenderness ratio (L/H): 11.20
  Aspect ratio (H/t):      1.00
  Total planar faces:      14
  Large planar faces:      2
  Max area:                462781.7
  Large face areas:        [462781.7, 462781.7]

Profile Type is : I
Determining NC1 Faces and Origin
Color name not defined. Use White by default
Many colors for color name CYAN, using first.
